In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"] #设 置 字 体
plt.rcParams["axes.unicode_minus"]=False #该 语 句 解 决 图 像中 的 “ -” 负 号 的 乱 码 问 题
%matplotlib inline
from scipy import stats
from scipy.special import boxcox1p
from scipy.stats import norm, skew
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn
import scipy.stats as stats
from scipy.stats import shapiro, skew
import math

# Re-loads all imports every time the cell is ran. 
%load_ext autoreload
%autoreload 2

from time import time

import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.5f}'.format

from IPython.display import display

# Sklearn tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Neural Networks
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers.csv_logs import CSVLogger

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt
df = pd.read_excel("../Data/新能源汽车产销量.xlsx")
df_date = df[['date', 'Production of new energy vehicles']]
df_date.set_index(df.date, inplace=True)
df_date.drop(columns='date', inplace=True)
test_percent = 0.1
test_point = np.round(len(df_date)*test_percent)
test_index = int(len(df_date) - test_point)
train = df_date.iloc[:test_index]
test = df_date.iloc[test_index:]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=2)
length = 6
generator = TimeseriesGenerator(scaled_train,scaled_train,
                               length=length,batch_size=1)


validation_generator = TimeseriesGenerator(scaled_test,scaled_test,
                                          length=length,batch_size=1)
n_features = 1
model = Sequential()
model.add(LSTM(12, input_shape = (length, n_features)))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mse')
model.fit_generator(generator, epochs = 10, 
                    validation_data = validation_generator, 
                    callbacks = [early_stop])
model.save('model_LSTM.h5')
loss = pd.DataFrame(model.history.history)
loss.plot()
prediction = []
evaluation_batch = scaled_train[-length:]
current_batch = evaluation_batch.reshape(1, length, n_features)
for i in range (len(test)):
    current_prediction = model.predict(current_batch)[0]
    prediction.append(current_prediction)
    current_batch = np.append(current_batch[:, 1:, :], [[current_prediction]], axis = 1)
prediction = scaler.inverse_transform(prediction)
test['LSTM Prediction'] = prediction
test.plot(figsize = (10, 4))
full_scaler = MinMaxScaler()
full_data_scale = scaler.transform(df_date)
length = 50 
generator = TimeseriesGenerator(full_data_scale, full_data_scale, length=length, batch_size=1)
model = Sequential()
model.add(LSTM(50, input_shape=(length, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit_generator(generator,epochs=6)
prediction = []
evaluation_batch = full_data_scale[-length:]
current_batch = evaluation_batch.reshape(1, length, n_features)
for i in range (120):
    current_prediction = model.predict(current_batch)[0]
    prediction.append(current_prediction)
    current_batch = np.append(current_batch[:, 1:, :], [[current_prediction]], axis = 1)
prediction = scaler.inverse_transform(prediction)

len(prediction)
from datetime import timedelta
from datetime import datetime
date_string = "2023-09"
date_format = "%Y-%m"
datetime_object = datetime.strptime(date_string, date_format)
future_months = [datetime_object + timedelta(days=30 * i) for i in range(1, 121)]
future_months
prediction_index = future_months
plt.plot(df_date.index,df_date['Production of new energy vehicles'])
plt.plot(prediction_index,prediction)
prelist = []
for i in prediction:
    prelist.append(i[0])
start_date = datetime(2023, 10, 1)

# 生成未来十年的日期列表（每个月）
date_list = [start_date + timedelta(days=30 * i) for i in range(10 * 12)]
date_list
forecast = pd.DataFrame(data={'forecast':prelist, 'Date': date_list})
forecast.to_csv("../result/prediction_lstm.csv")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
pd.options.display.float_format = '{:.2f}'.format
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import pickle

from itertools import combinations
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima.model import ARIMA as ARIMA
import statsmodels.api as sm
import statsmodels.tsa.api as smt
def test_stationarity(timeseries):
    #Determing rolling statistics
    MA = timeseries.rolling(window=12).mean()
    MSTD = timeseries.rolling(window=12).std()

    #Plot rolling statistics:
    plt.figure(figsize=(15,5))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(MA, color='red', label='Rolling Mean')
    std = plt.plot(MSTD, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)

    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
def tsplot(y, lags=None, figsize=(12, 7), style='bmh'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        
        y.plot(ax=ts_ax)
        p_value = sm.tsa.stattools.adfuller(y)[1]
        ts_ax.set_title('Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}'.format(p_value))
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()
dec = sm.tsa.seasonal_decompose(df_date['Production of new energy vehicles'],period = 12,model = 'additive').plot()
plt.show()

test_stationarity(df_date['Production of new energy vehicles'])
#### Step 3  差分转平稳
def stationarity(timeseries): #平稳性处理（timeseries 时间序列）
    ## 差分法,保存成新的列
    diff1 = timeseries.diff(1).dropna()  # 1阶差分 dropna() 删除缺失值
    diff2 = diff1.diff(1).dropna() #在一阶差分基础上再做一次一阶差分，即二阶查分
    ## 画图
    diff1.plot(color = 'red',title='diff 1',figsize=(10,4))
    diff2.plot(color = 'black',title='diff 2',figsize=(10,4))

    
    ## 平滑法
    rollmean = timeseries.rolling(window=4,center = False).mean() ## 滚动平均
    rollstd = timeseries.rolling(window=4,center = False).std() ## 滚动标准差
    ## 画图 
    rollmean.plot(color = 'yellow',title='Rolling Mean',figsize=(10,4))
    rollstd.plot(color = 'blue',title='Rolling Std',figsize=(10,4))
    
    return diff1,diff2,rollmean,rollstd
diff1,diff2,rollmean,rollstd = stationarity(df_date)
tsplot(diff2['Production of new energy vehicles'])
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import adfuller as ADF
from statsmodels.stats.diagnostic import acorr_ljungbox

import statsmodels.api as sm


def testwhitenoise(data):
    m = 10# 检验10个自相关系数
    acf,q,p = sm.tsa.acf(data,nlags=m,qstat=True)
    out = np.c_[range(1,m+1),acf[1:],q,p]
    output = pd.DataFrame(out,columns=['lag','自相关系数','统计量Q值','p_values'])
    output = output.set_index('lag')# 设置第一列索引名称,可省略重复索引列1
    print(output)

def teststeady(data,count=0):
    res_ADF = ADF(data)
    print('ADF检验结果为：', res_ADF)
    Pv = res_ADF[1]
    if Pv > 0.05:
        print('\033[1;31mP值：%s，原始序列不平稳，要进行差分！\033[0m' % round(Pv,5))
        count = count + 1
        print('\033[1;32m进行了%s阶差分后的结果如下\033[0m' % count)
        data = data.diff(1).dropna()
        teststeady(data,count)
    else:
        print('\033[1;34mP值：%s，原始序列平稳，继续建模\033[0m'% round(Pv,5))
    return data
testwhitenoise(diff2)
teststeady(diff2)

def confirm_p_q(data):
    fig = plt.figure(figsize=(8,6))
    testwhitenoise(data)
    train = teststeady(data)
    ax1 = fig.add_subplot(211)
    fig = sm.graphics.tsa.plot_pacf(train, lags=10, ax=ax1)
    ax2 = fig.add_subplot(212)
    fig = sm.graphics.tsa.plot_acf(train, lags=10, ax=ax2)
    plt.show()  ###可视化定阶

    pmax = int(len(data) / 10)
    qmax = int(len(data) / 10)
    AIC = sm.tsa.arma_order_select_ic(train,max_ar=pmax,max_ma=qmax,ic='aic')['aic_min_order']
    BIC = sm.tsa.arma_order_select_ic(train,max_ar=pmax,max_ma=qmax,ic='bic')['bic_min_order']
    HQIC = sm.tsa.arma_order_select_ic(train,max_ar=pmax,max_ma=qmax,ic='hqic')['hqic_min_order']
    print('AIC：',AIC)
    print('BIC：',BIC)
    print('HQIC：',HQIC)
    return AIC
pq = confirm_p_q(diff2)##返回p,q值

from statsmodels.tsa.arima.model import ARIMA as ARIMA
model = ARIMA(diff2['Production of new energy vehicles'], order = (10,2,7))
model_fit = model.fit()
print(model_fit.summary())
def prediction(data):
    ##tempmodel = ARMA(teststeady(data),pq).fit(disp=-1)
    tempmodel = sm.tsa.arima.ARIMA(data, order=(2,2,7)).fit()
    print(tempmodel.summary())
    #num = 10
    #predictoutside1 = tempmodel.forecast(num)[0]#预测样本外的
    predictoutside2 = tempmodel.predict(len(tempmodel.predict()),len(tempmodel.predict()) + 9,dynamic=True)##也是样本外预测，预测结果一致
    predictinside = tempmodel.predict()##样本内预测
    init_value = diff2.values[0]

    fig = plt.figure(figsize=(10, 6))
    predictinside = predictinside.cumsum()##差分还原
    pretrueinside = init_value + predictinside
    startprevalue = list(pretrueinside)[-1]
    predictoutside2 = predictoutside2.cumsum()##差分还原
    pretrueoutside = startprevalue + predictoutside2
    
    ##作图
    plt.plot(diff2.values,label='Original')
    plt.plot([init_value] + list(pretrueinside),label='Prediction')
    X = [i for i in range(len(diff2)-11,len(diff2))]
    #plt.plot(X,[startprevalue] + list(pretrueoutside), label='样本外预测值')
    allpredata = [init_value] + list(pretrueinside) + list(pretrueoutside)
    plt.legend()
    plt.show()
    return tempmodel,allpredata
preres = prediction(diff2)

from sklearn.metrics import mean_squared_error

# 实际值
actual_values = diff2.values

# 预测值
predicted_values = preres[1]

# 计算均方误差
mse = mean_squared_error(actual_values, predicted_values[:79])
print(f"均方误差 (MSE): {mse}")

from sklearn.metrics import r2_score

# 计算R-square
r2 = r2_score(predicted_values[:79], actual_values)
print(f"R-square: {r2}")
tempmodel = sm.tsa.arima.ARIMA(df_date, order=(2,2,7)).fit()
forecast = tempmodel.predict(start='2023-10-1',end='2033-09-1')
start_date = datetime(2023, 10, 1)

# 生成未来十年的日期列表（每个月）
date_list = [start_date + timedelta(days=30 * i) for i in range(10 * 12)]
forecast = pd.DataFrame(data={'Date':date_list, 'Prediction': forecast})
plt.plot(forecast.Date, forecast.Prediction)
forecast.to_csv("../result/Question2/ARIMAPredction.csv", index=False)